In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,48484.99,-0.02,-0.03,-0.23,0.02,0.01,0.02,-0.01,-0.00
3,-0.02,9457.02,0.52,-0.02,0.06,-0.00,0.00,0.00,0.00
4,-0.03,0.52,2095.68,0.00,0.02,-0.00,0.00,-0.00,0.00
5,-0.23,-0.02,0.00,347.07,0.01,0.01,-0.00,-0.00,-0.00
6,0.02,0.06,0.02,0.01,95.16,0.00,-0.00,0.00,-0.00
7,0.01,-0.00,-0.00,0.01,0.00,27.17,0.00,-0.00,-0.00
8,0.02,0.00,0.00,-0.00,-0.00,0.00,10.96,-0.00,0.00
9,-0.01,0.00,-0.00,-0.00,0.00,-0.00,-0.00,1.82,-0.00
10,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00000,-0.00000,-0.00006,0.00001,0.00001,0.00002,-0.00004,-0.00004
3,-0.00000,1.00000,0.00012,-0.00001,0.00007,-0.00001,0.00001,0.00001,0.00004
4,-0.00000,0.00012,1.00000,0.00000,0.00005,-0.00002,0.00000,-0.00003,0.00003
5,-0.00006,-0.00001,0.00000,1.00000,0.00003,0.00006,-0.00004,-0.00005,-0.00004
6,0.00001,0.00007,0.00005,0.00003,1.00000,0.00002,-0.00004,0.00001,-0.00004
7,0.00001,-0.00001,-0.00002,0.00006,0.00002,1.00000,0.00002,-0.00001,-0.00003
8,0.00002,0.00001,0.00000,-0.00004,-0.00004,0.00002,1.00000,-0.00003,0.00001
9,-0.00004,0.00001,-0.00003,-0.00005,0.00001,-0.00001,-0.00003,1.00000,-0.00005
10,-0.00004,0.00004,0.00003,-0.00004,-0.00004,-0.00003,0.00001,-0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.8182498515895414

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.61368830e+08 9.78640889e+06 4.70454713e+05 1.25817534e+04
 9.08418464e+02 7.03451203e+01 9.86461982e+00 2.07110013e-01
 3.69448081e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999665,-0.025316,-0.005383,-0.000894,-0.000244,-0.000069,-0.000028,-0.000005,-5.433441e-07
1,0.025118,0.999115,-0.033312,-0.005229,-0.001425,-0.000409,-0.000164,-0.000027,-2.525383e-06
2,0.006187,0.032991,0.998998,-0.028530,-0.007579,-0.002165,-0.000872,-0.000145,-1.393239e-05
3,0.001179,0.006027,0.027799,0.997706,-0.059113,-0.015464,-0.006214,-0.001033,-1.022453e-04
4,0.000385,0.001970,0.008922,0.057295,0.994857,-0.077773,-0.028680,-0.004679,-4.631728e-04
5,0.000131,0.000672,0.003036,0.018390,0.071242,0.984557,-0.157453,-0.020676,-2.031659e-03
6,0.000076,0.000390,0.001764,0.010665,0.039208,0.152347,0.983438,-0.088963,-8.413449e-03
7,0.000019,0.000095,0.000430,0.002596,0.009455,0.033184,0.083245,0.990951,-9.944019e-02
8,0.000004,0.000019,0.000085,0.000516,0.001879,0.006577,0.016300,0.098238,9.950058e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00033543, 0.00088547, 0.0010023 , 0.0022945 , 0.00514301,
       0.01544252, 0.01656162, 0.00904874, 0.00499422])